<a href="https://colab.research.google.com/github/arindampal2020/arindampal2020/blob/main/Beam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!{'pip uninstall apache_beam'}

In [ ]:
!{'pip install apache_beam'}

In [ ]:
! cd data

In [1]:
from google.colab import files
uploaded = files.upload()

Saving store_sales.csv to store_sales.csv


In [ ]:
import apache_beam as beam
def str_rep (line):
  b = {'(': '' , ')':'', '\t': '' , ':':'' }
  for i,j in b.items() : 
    line = line.replace(i,j)
  return line

p1 = beam.Pipeline()
wordcount = (
    p1
    |beam.io.ReadFromText('./data/data.txt')
    |beam.FlatMap(lambda rec: rec.split("\t"))
    |beam.FlatMap(lambda lines: lines.upper().split(" "))
    |beam.Map(str_rep)
    |beam.Filter(lambda word: word != '')
    |beam.Map(lambda word: (word , 1))
    |beam.CombinePerKey(sum)
    |beam.MapTuple(lambda word, count: '%s , %s' % (word, count))
    |beam.io.WriteToText('output1.txt')
)
p1.run()

In [124]:
import apache_beam as beam

def format_row(elements):
  Store_id, Store_location, Product_id, Product_category, number_of_pieces_sold, buy_rate, sell_price, unix_timestamp  = elements.split(',')
  profit = float(sell_price)*int(number_of_pieces_sold)- float(buy_rate)*int(number_of_pieces_sold)
  return Store_id, Store_location, Product_id, Product_category, number_of_pieces_sold, buy_rate,  sell_price , profit

def show_results(key , total_profit):
  #location , total_profit = elements.split(',')
  #for k,v in key:
  keys=list(key)
  print(' Location -> ' + keys[0] + ' Product ID -> '+ keys[1] + ' Total Profit-> ' + str(total_profit) )
  return keys[0] + ',' + keys[1] +',' + str(total_profit)

class show_counts(beam.DoFn):
  def process(self,rows , text):
    row=list(rows)
    print('Product-> '+ str(row[0])+ ' ' + text +'  ->' + str(row[1]) )

class process_counts (beam.DoFn):
  def process (self , elements):
      (key, values) = elements 
      #print(str(elements))  
      return [(key, sum(values))]

class AverageFn(beam.CombineFn):
  
  def create_accumulator(self):
     return (0.0, 0)   # initialize (sum, count)

  def add_input(self, sum_count, input):
    (sum, count) = sum_count
    return sum + input, count + 1

  def merge_accumulators(self, accumulators):
    
    ind_sums, ind_counts = zip(*accumulators)       
    return sum(ind_sums), sum(ind_counts)        # (84,8)

  def extract_output(self, sum_count):    
    
    (sum, count) = sum_count    # combine globally using CombineFn
    return round(sum / count,2) if count else float('NaN')
  

p = beam.Pipeline()

read_data =(
    p
    |'Read File'>>beam.io.ReadFromText('store_sales.csv' , skip_header_lines=1)
    |'Format FIle'>>beam.Map(format_row)
    |'Filter Row'>>beam.Filter(lambda row : row[1]=='Mumbai' or row[1]=='Bangalore')    
)
profit = (
    read_data
    |'Map Profit'>>beam.Map(lambda row : ((row[1] , row[3]) , row[7]))
    |'Total Profit'>>beam.CombinePerKey(sum)
    |'Show Profits'>>beam.MapTuple(show_results)
    |'Write Profits'>>beam.io.WriteToText('Profits.csv')
)
Product_Counts= (
    read_data
    |'Map Products'>>beam.Map(lambda row : ((row[3]) ,1))
    |'Group By'>>beam.GroupByKey()
    |'Products Count'>>beam.ParDo(process_counts())
    #|'Show Product Counts'>>beam.ParDo(show_counts() , 'Counts')
    |beam.Map(lambda row : str(row[0]) + ',' + str(row[1]))
    |'Write Products Count'>>beam.io.WriteToText('ProductCounts.csv')
)

Average_profits=(
    read_data
    |'Map Avg'>>beam.Map(lambda row : (row[3] , row[7]))
    |'Average Profit'>>beam.CombinePerKey(AverageFn())
    #|'Show Average'>>beam.ParDo(show_counts(), 'Average')
    |beam.Map(lambda row : str(row[0]) + ',' + str(row[1]))
    |'Write Average'>>beam.io.WriteToText('Average.csv')
)
p.run()

 Location -> Mumbai Product ID -> Cosmetics Total Profit-> 5122.0
 Location -> Mumbai Product ID -> Education Total Profit-> 4389.0
 Location -> Mumbai Product ID -> Electronics Total Profit-> 3794.0
 Location -> Bangalore Product ID -> Groceries Total Profit-> 2143.0
 Location -> Bangalore Product ID -> Education Total Profit-> 3149.0
 Location -> Bangalore Product ID -> Cosmetics Total Profit-> 1866.0
 Location -> Bangalore Product ID -> Kitchen Total Profit-> 3199.0
 Location -> Mumbai Product ID -> Groceries Total Profit-> 3817.0
 Location -> Mumbai Product ID -> Fashion Total Profit-> 4533.0
 Location -> Mumbai Product ID -> Kitchen Total Profit-> 4322.0
 Location -> Bangalore Product ID -> Electronics Total Profit-> 4469.0
 Location -> Bangalore Product ID -> Fashion Total Profit-> 2939.0


In [122]:
!ls -lrt 

total 56
drwxr-xr-x 1 root root  4096 Nov  1 13:35 sample_data
-rw-r--r-- 1 root root 25555 Nov 19 01:46 store_sales.csv
drwxr-xr-x 2 root root  4096 Nov 19 05:06 beam-temp-Profitls.csv-7e272c8848f611ec87cb0242ac1c0002
drwxr-xr-x 2 root root  4096 Nov 19 05:06 beam-temp-Average.csv-7e309e4e48f611ec87cb0242ac1c0002
-rw-r--r-- 1 root root   302 Nov 19 05:07 Profitls.csv-00000-of-00001
-rw-r--r-- 1 root root   302 Nov 19 05:10 Profits.csv-00000-of-00001
-rw-r--r-- 1 root root    99 Nov 19 05:10 Average.csv-00000-of-00001
-rw-r--r-- 1 root root    76 Nov 19 05:10 ProductCounts.csv-00000-of-00001


In [125]:
!head -20 Average.csv-00000-of-00001

Cosmetics,112.71
Education,117.78
Electronics,131.16
Groceries,110.37
Kitchen,144.63
Fashion,133.43
